# Emotion Chat Bot

In [1]:
import os.path
import uuid
from random import randint

import torch
from torch import Tensor
from transformers import (
	AutoModelForSequenceClassification,
	AutoTokenizer,
	BitsAndBytesConfig,
	GenerationConfig,
	TextClassificationPipeline,
	TextStreamer,
)
from unsloth import FastLanguageModel

from src.models.libs.EmotionChatBot import create_candidates_buffer
from src.models.libs.EmotionTransition import EmotionModel, EmotionPresentationSimilarityAnalyser, \
	generate_dummy_representation, get_emotion_composition
from src.models.libs.ResponseGenerationPipeline import ResponseGeneratorPipeline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load Each Module

### Response Generator

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
	model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
	attn_implementation="flash_attention_2",
	pretraining_tp=1,
	load_in_4bit=True,
	device_map="auto",
	low_cpu_mem_usage=True,
	trust_remote_code=True,
)
tokenizer.padding_side = "left"
tokenizer.clean_up_tokenization_spaces = True

In [ ]:
response_generator = ResponseGeneratorPipeline(
	model,
	tokenizer,
	framework="pt",
	task="conversation-generation",
	num_workers=16,
	torch_dtype="auto",
	add_special_tokens=True,
	truncation=False,
	padding=True,
)

In [ ]:
FastLanguageModel.for_inference(response_generator.model)

### Sentiment Analyzer

In [3]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
	"Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
	quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16),
	device_map="auto",
	low_cpu_mem_usage=True,
)

In [4]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
	"Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
	trust_remote_code=True,
)

In [5]:
sentiment_analyzer = TextClassificationPipeline(
	model=sentiment_analysis_model,
	tokenizer=sentiment_analysis_tokenizer,
	framework="pt",
	task="sentiment-analysis",
	num_workers=12,
	torch_dtype="auto",
)

In [17]:
sentiment_analyzer("Hello, how are you?", return_all_scores=True)[0]

[{'label': 'neutral', 'score': 0.690626323223114},
 {'label': 'anger', 'score': 0.00045183763722889125},
 {'label': 'disgust', 'score': 0.00014441600069403648},
 {'label': 'fear', 'score': 0.00017488074081484228},
 {'label': 'happiness', 'score': 0.3064635992050171},
 {'label': 'sadness', 'score': 0.0008070625481195748},
 {'label': 'surprise', 'score': 0.0013319212011992931}]

In [28]:
a = sentiment_analyzer("Goodbye!", return_all_scores=True, function_to_apply="softmax")[0]
a

[{'label': 'neutral', 'score': 0.052009064704179764},
 {'label': 'anger', 'score': 0.01242279727011919},
 {'label': 'disgust', 'score': 0.0010472259018570185},
 {'label': 'fear', 'score': 0.0029227114282548428},
 {'label': 'happiness', 'score': 0.9245875477790833},
 {'label': 'sadness', 'score': 0.0017131471540778875},
 {'label': 'surprise', 'score': 0.005297517869621515}]

In [29]:
sum([i["score"] for i in a])

1.0000000121071935

In [50]:
b = torch.tensor([i["score"] for i in a]).softmax(dim=-1)
b

tensor([0.1225, 0.1178, 0.1164, 0.1167, 0.2932, 0.1165, 0.1169])

In [51]:
b.argmax()

tensor(4)

In [32]:
sum(torch.tensor([i["score"] for i in a]).softmax(dim=0))

tensor(1.)

In [49]:
torch.clamp(torch.rand((1, 7), dtype=torch.float32), -1, 1)

tensor([[0.2914, 0.4428, 0.3310, 0.8346, 0.0598, 0.7446, 0.0218]])

### Emotion Predictor

In [37]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
	"Shotaro30678/emotion_predictor_for_emotion_chat_bot",
	quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16),
	device_map="auto",
	low_cpu_mem_usage=True,
)

In [ ]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
	"Shotaro30678/emotion_predictor_for_emotion_chat_bot",
	trust_remote_code=True,
)

In [ ]:
emotion_predictor = TextClassificationPipeline(
	model=emotion_predictor_model,
	tokenizer=emotion_predictor_tokenizer,
	framework="pt",
	task="sentiment-analysis",
	num_workers=12,
	torch_dtype="auto",
)

In [ ]:
emotion_predictor("Hello, how are you?", return_all_scores=False)

### Emotion Model

In [ ]:
emotion_model = EmotionModel.from_pretrained("hermeschen1116/emotion_model_for_emotion_chat_bot")

### Similarity Analyzer

In [ ]:
threshold: float = 0.7

In [ ]:
similarity_analyzer = EmotionPresentationSimilarityAnalyser(None, threshold=threshold)

## Combine All Modules

In [ ]:
emotions: list = [
	"neutral",
	"anger",
	"disgust",
	"fear",
	"happiness",
	"sadness",
	"surprise",
]

### Initialize

In [ ]:
chat_buffer_size: int = 10

In [ ]:
logfile_uuid: uuid.UUID = uuid.uuid4()
if not os.path.isdir("./logs"):
	os.mkdir("./logs")

In [ ]:
generation_config = GenerationConfig(
	max_new_tokens=20,
	min_new_tokens=5,
	repetition_penalty=1.5,
	pad_token_id=tokenizer.pad_token_id,
	eos_token_id=tokenizer.eos_token_id,
	use_cache=True,
)

In [ ]:
streamer = TextStreamer(tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
bot_ideal_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
similarity_analyzer.ideal_emotion_representation = bot_ideal_emotion_representation
dict(zip(emotions, bot_ideal_emotion_representation.tolist()))

In [ ]:
bot_emotion_id: int = randint(0, 6)
bot_emotion_representation: Tensor = generate_dummy_representation(bot_emotion_id)
bot_emotion_representation

In [ ]:
system_prompt: str = input("Enter your system prompt: ").strip()
bot_response: dict = {"emotion": emotions[bot_emotion_id], "dialog": "(Talk to me...)"}

In [ ]:
chat_buffer: list = [{"role": "system", "content": {"emotion": "", "dialog": system_prompt}}]

In [ ]:
chat_buffer

In [ ]:
while True:
	user_response: str = input(f"Bot({bot_response["emotion"]}): {bot_response["dialog"]}").strip()
	if user_response == "quit":
		break

	sentiment_analysis_result: list = sentiment_analyzer(user_response, return_all_scores=True)
	user_emotion_composition: Tensor = get_emotion_composition(sentiment_analysis_result[0])
	user_emotion: str = emotions[user_emotion_composition.argmax()]
	chat_buffer.append(
		{
			"role": "user",
			"content": {
				"emotion": user_emotion,
				"dialog": user_response,
			},
		}
	)

	bot_emotion_representation = emotion_model.forward(user_emotion_composition, bot_emotion_representation)

	bot_chat_simulations: list = []
	while True:
		bot_chat_simulations = create_candidates_buffer(chat_buffer)
		bot_chat_simulations = [
			chat[0]
			for chat in response_generator(bot_chat_simulations, generation_config=generation_config, streamer=streamer)
		]
		bot_chat_simulations = list(filter(lambda chat: chat[-1]["content"]["dialog"] != "", bot_chat_simulations))
		bot_chat_simulations = list(
			filter(lambda chat: chat[-1]["content"]["dialog"].endswith((".", "!", "?")), bot_chat_simulations)
		)
		if len(bot_chat_simulations) != 0:
			break

	bot_response_simulations: list = [chat[-1]["content"] for chat in bot_chat_simulations]

	user_future_emotion_composition_simulations: dict = {
		response["emotion"]: get_emotion_composition(emotion_predictor(response["dialog"], return_all_scores=True)[0])
		for response in bot_response_simulations
	}

	bot_future_emotion_representations: dict = {
		k: emotion_model.forward(v, bot_emotion_representation)
		for k, v in user_future_emotion_composition_simulations.items()
	}

	emotion_representation_similarity_scores: list = similarity_analyzer(
		torch.stack(list(bot_future_emotion_representations.values())),
	)

	candidate_emotion_index: int = torch.argmax(emotion_representation_similarity_scores).item()
	bot_best_response_emotion: str = emotions[candidate_emotion_index]

	if len(chat_buffer) == chat_buffer_size + 1:
		chat_buffer.pop(1)
	chat_buffer.append({"role": "bot", "content": {"emotion": bot_best_response_emotion, "dialog": ""}})

	chat_buffer = response_generator(chat_buffer, generation_config=generation_config, streamer=streamer)[0]
	print()
	if chat_buffer[-1]["content"]["dialog"] == "":
		chat_buffer[-1]["content"]["dialog"] = bot_response_simulations[candidate_emotion_index]["dialog"]
	bot_response = chat_buffer[-1]["content"]